# 导入所需库

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from joblib import Parallel, delayed
from sklearn.metrics import mean_squared_error

# 导入数据

In [ ]:

train_rain = pd.read_csv('./A-雨量水位（2014-2019）.csv')
train_runoff = pd.read_csv('./A-入库流量（2014-2019）.csv')
test_rain = pd.read_csv('./A-雨量水位（2020-2021）.csv')

# 数据预处理

In [ ]:

#去除无关列
train_rain = train_rain.drop(columns=['NAME','S','AVGS','MAXS','MINS','SPAN'])
train_runoff = train_runoff.drop(columns=['NAME','S','AVGS','MAXS','MINS','SPAN'])
test_rain = test_rain.drop(columns=['NAME','S','AVGS','MAXS','MINS','SPAN'])

In [ ]:

#由此可知训练集和测试集站点数量不一样
print(f"{train_rain.nunique()}\n{test_rain.nunique()}")
#先将站点转换为列例如210254，再取相同的站点保存
# 列出所有时间列
time_columns = ['TIME', 'MAXT','MINT']

# 批量转换时间列为 datetime 类型
for col in time_columns:
    train_rain[col] = pd.to_datetime(train_rain[col])
    # 展开雨量数据
train_rain = train_rain.pivot_table(index='TIME',
                              columns='SENID',
                              values='V',
                              aggfunc='first')
train_rain

In [ ]:
#测试集一样处理
# 列出所有时间列
time_columns = ['TIME', 'MAXT','MINT']

# 批量转换时间列为 datetime 类型
for col in time_columns:
    test_rain[col] = pd.to_datetime(test_rain[col])
test_rain = test_rain.pivot_table(index='TIME',
                              columns='SENID',
                              values='V',
                              aggfunc='first')
test_rain

In [ ]:
#取相同的站点
train_rain_colmns = train_rain.columns
test_rain_colmns = test_rain.columns
common_colmns = train_rain_colmns.intersection(test_rain_colmns)
train_rain = train_rain[common_colmns]
test_rain = test_rain[common_colmns]

# 特征工程

In [ ]:

#查看数据的缺失值情况
missing_train_v = (train_rain.isnull().sum()/len(train_rain))*100
missing_train_v

In [ ]:

# 设置缺失值百分比的阈值
threshold = 90  # 例如，设置阈值为90%
# 找出缺失值百分比高于阈值的列
columns_to_drop = missing_train_v[missing_train_v > threshold].index
columns_to_drop

In [ ]:

#删除超过阈值的列
for col in columns_to_drop:
    train_rain = train_rain.drop(columns=[col])
for col in columns_to_drop:
    test_rain = test_rain.drop(columns=[col])

In [ ]:

train_rain.describe()
#发现有很离谱的负数

In [ ]:
def handle_rain_outliers(series, negative_threshold=-100):
    """
    雨量数据专用异常处理：
    1. 保留所有 >=0 的值
    2. 将微小负值（>-100）修正为0
    3. 将极端负值（<=-100）视为缺失值
    4. 线性插值填充缺失值
    """
    # 复制数据避免修改原始Series
    s = series.copy()
    
    # 步骤1: 标记需要处理的异常点
    neg_mask = s < 0
    
    # 步骤2: 微小负值设为0
    s.loc[(s >= negative_threshold) & (s < 0)] = 0
    
    # 步骤3: 极端负值设为NaN
    s.loc[s < negative_threshold] = np.nan
    
    return s

# 应用处理
train_rain = train_rain.apply(handle_rain_outliers)


In [ ]:
# 使用前向填充+线性插值组合
train_rain = train_rain.resample("15T").asfreq()
train_rain = train_rain.ffill().interpolate(method='linear')
print("\n线性插值后的数据：")
print(train_rain)

In [ ]:
# 添加总雨量列（所有站点之和）
train_rain['count'] = train_rain.sum(axis=1)

# 验证结果
print("\n添加总雨量列后的数据示例：")
print(train_rain[['count']].head())  # 只显示count列的前5行

In [ ]:

# 处理所有列的异常值
test_rain = test_rain.apply(handle_rain_outliers)

print("\n处理异常值后的数据：")
print(test_rain)
# 使用前向填充+线性插值组合
test_rain = test_rain.resample("15T").asfreq()
test_rain = test_rain.ffill().interpolate(method='linear')

print("\n线性插值后的数据：")
print(test_rain)


In [ ]:
# 添加总雨量列（所有站点之和）
test_rain['count'] = test_rain.sum(axis=1)

In [ ]:


plt.plot(train_rain.index,train_rain['count'])
plt.xlabel('Time')
plt.ylabel('V')
plt.title('Line Chart of Long Time Series Data')
plt.show()

In [ ]:

train_runoff.describe()

In [ ]:

train_runoff['TIME'] =pd.to_datetime(train_runoff['TIME'])
train_runoff.set_index('TIME',inplace=True)

# 使用前向填充+线性插值组合
train_runoff = train_runoff.resample("15T").asfreq()
train_runoff = train_runoff.ffill().interpolate(method='linear')
plt.figure(figsize=(10,6))
plt.plot(train_runoff.index,train_runoff['V'])
plt.title('Runoff Over Time')
plt.xlabel('Time')
plt.ylabel('Runoff (m^3/s)')
plt.show()

In [ ]:

merged_data = pd.merge(train_rain,train_runoff,on='TIME',how = 'inner')
merged_data

In [ ]:

# 计算每个特征与目标变量之间的皮尔逊相关系数
correlation_matrix = merged_data.corr()
print("Correlation matrix:")
print(correlation_matrix)

# 提取目标变量与其他特征的相关系数
target_correlation = correlation_matrix['V'].drop('V')
print("\nCorrelation with target variable:")
print(target_correlation)
a = target_correlation.sort_values(ascending=False)
a.head(25)

In [ ]:

# 标记训练和测试数据
merged_data['is_test'] = 0
test_rain['is_test'] = 1
test_rain['V'] = np.nan  # 添加缺失的径流列

# 合并数据并按时间排序
full_data = pd.concat([merged_data, test_rain]).sort_values('TIME')

In [ ]:


fixed_columns = ['V', 'SENID', 'is_test', 'AVGV', 'MAXV','MAXT','MINV','MINT','TIME']
sites = [col for col in full_data.columns 
        if col not in fixed_columns 
        and str(col).isdigit()]

# 扩展滞后范围（包含6小时到24小时）
lags = [24, 48, 96]  # 对应6小时、12小时、24小时
for site in sites:
    for lag in lags:
        full_data[f'{site}_lag{lag}'] = full_data[site].shift(lag)
# 添加过去6小时统计特征
window_size = 24  # 6小时窗口
for site in sites:
    full_data[f'{site}_mean_6h'] = full_data[site].rolling(window=window_size).mean()
    full_data[f'{site}_max_6h'] = full_data[site].rolling(window=window_size).max()
# 添加时间特征（月份）
full_data['month'] = full_data.index.month
full_data['month_sin'] = np.sin(2 * np.pi * full_data['month'] / 12)
full_data['month_cos'] = np.cos(2 * np.pi * full_data['month'] / 12)
full_data['hour'] = full_data.index.hour
full_data['week'] = full_data.index.isocalendar().week
full_data['hour_sin'] = np.sin(2 * np.pi * full_data['hour'] / 24)
full_data['hour_cos'] = np.cos(2 * np.pi * full_data['hour'] / 24)

In [ ]:

# 重新分割训练集和测试集
train_processed = full_data[full_data['is_test'] == 0]
test_processed = full_data[full_data['is_test'] == 1]

# 定义特征列
feature_cols = sites + [f'{site}_lag{lag}' for site in sites for lag in lags] + ['month_sin', 'month_cos','hour_sin','hour_cos']
X_train = train_processed[feature_cols]
y_train = train_processed['V']
X_test = test_processed[feature_cols]

In [ ]:
# 计算每个特征与目标变量之间的皮尔逊相关系数
correlation_matrix = train_processed.corr()
print("Correlation matrix:")
print(correlation_matrix)

# 提取目标变量与其他特征的相关系数
target_correlation = correlation_matrix['V'].drop('V')
print("\nCorrelation with target variable:")
print(target_correlation)
a = target_correlation.sort_values(ascending=False)
a.head(25)

In [ ]:
#筛选特征
X = X_train  # 所有特征
y = y_train 

# 使用时序交叉验证
tscv = TimeSeriesSplit(n_splits=3)

# 特征重要性存储
feature_importance = pd.DataFrame(index=X.columns)

for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # 训练LGBM模型
    model = lgb.LGBMRegressor(
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        random_state=42
    )
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],)
    
    # 记录特征重要性（按信息增益）
    fold_importance = pd.Series(model.feature_importances_, 
                               index=X.columns)
    feature_importance[f'fold_{fold}'] = fold_importance

# 计算平均重要性
feature_importance['mean'] = feature_importance.mean(axis=1)
feature_importance = feature_importance.sort_values('mean', ascending=False)

# 选择Top70特征
selected_features = feature_importance.head(70).index.tolist()
print("Selected Features (LGBM):\n", selected_features)

In [ ]:
X_test = X_test.loc[:,selected_features]
X_train = X_train.loc[:,selected_features]
X_val = X_val.loc[:,selected_features]

In [ ]:
#标准化
scaler = StandardScaler()
X_test.columns = X_test.columns.astype(str)
X_train.columns = X_train.columns.astype(str)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# 验证集标准化（必须用transform）
X_val.columns = X_val.columns.astype(str)
X_val_scaled = scaler.transform(X_val)  # 直接应用训练集的均值和标准差

# 模型训练和预测

In [ ]:
# 定义模型参数
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'min_data_in_leaf': 20,
    'feature_fraction': 0.8,
    'bagging_freq': 1,
    'bagging_fraction': 0.8,
    'verbosity': -1
}

# 创建数据集
train_data = lgb.Dataset(X_train_scaled, label=y_train)

# 训练模型
model = lgb.train(params,
                 train_data,
                 num_boost_round=1000,
                 valid_sets=[train_data])




In [ ]:

test_processed['V'] = model.predict(X_test_scaled)
test_processed['V']

 # 结果保存

In [ ]:
# 修正结果采样方式（保留所有预测点）
predictions = test_processed['V']

# 结果后处理（移动平均平滑）
predictions = predictions.rolling(window=4, min_periods=1).mean()

# 使用列表切片来获取每四个数据中的一个
predictions = predictions[::4]

print(predictions)
predictions.to_csv('predicted_last.csv', index=False)